# Microsoft Semantic Kernel With IBM Granite 3.3 LLM

> **⚠️ Important Note**: This notebook is **not compatible with Google Colab** as it requires a local Ollama installation running a Granite model. Please run this notebook in a local environment with Ollama properly configured and running on port 11434.

## What is Microsoft Semantic Kernel?

**Microsoft Semantic Kernel (SK)** is an open-source SDK that allows developers to integrate AI capabilities into their applications through a unified framework. It acts as a middleware layer that orchestrates AI models, plugins, and traditional programming logic.

## Key Features & Differentiators

### 🔌 **Plugin Architecture**
- **Semantic Kernel**: Uses a sophisticated plugin system where AI functions can be chained together with traditional code
- **Other Frameworks**: Often focus on single model interactions or require custom integration work

### 🧠 **AI Orchestration** 
- **Semantic Kernel**: Built-in planning capabilities that can automatically sequence multiple AI operations
- **LangChain/LlamaIndex**: Primarily focused on chaining operations manually
- **OpenAI API**: Direct model access without orchestration features

### 🔄 **Multi-Model Support**
- **Semantic Kernel**: Vendor-agnostic - works with OpenAI, Azure OpenAI, Hugging Face, Ollama, and more
- **Other Frameworks**: Often tied to specific providers or require extensive configuration

### 🎯 **Enterprise-Ready**
- **Semantic Kernel**: Built with enterprise needs in mind - security, scalability, and integration with Microsoft ecosystem
- **Other Frameworks**: May require additional tooling for enterprise deployment

## About Granite 3.3

**IBM Granite 3.3** models feature enhanced reasoning capabilities and support for Fill-in-the-Middle (FIM) code completion. Key highlights include:

- **Enhanced Architecture**: New dense architecture trained with 12 trillion tokens across 12 languages and 116 programming languages
- **128K Context Length**: Extended context for complex tasks
- **Strong RAG Performance**: Excellent retrieval-augmented generation capabilities
- **Function Calling**: Native support for tool/function calling
- **Response Controls**: Built-in length and originality controls
- **Apache 2.0 License**: Fully open-source

## Prerequisites

Before running this notebook, ensure you have:

1. **Ollama installed locally**: Download and install from [https://ollama.ai](https://ollama.ai)
2. **Granite 3.3 8B model downloaded**: Run `ollama pull granite3.3:8b` in your terminal
3. **Ollama running**: Start Ollama service (usually `ollama serve` or it starts automatically)
4. **Python 3.10, 3.11, or 3.12**: This notebook requires a recent Python version

## This Demo

This notebook demonstrates a simple chatbot using Semantic Kernel with:
- **Ollama** as the local AI provider
- **Granite 3.3 8B** model for responses  
- **Granite 3.3 prompt template** following official guidelines
- **Async execution** for responsive interactions

In [16]:
# Install and import the granite community utils
%pip install git+https://github.com/ibm-granite-community/utils

  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/jq/jnw70_td671fdvqt9w41w6r00000gn/T/pip-req-build-mtmwf48_
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/jq/jnw70_td671fdvqt9w41w6r00000gn/T/pip-req-build-mtmwf48_
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/jq/jnw70_td671fdvqt9w41w6r00000gn/T/pip-req-build-mtmwf48_
  Resolved https://github.com/ibm-granite-community/utils to commit da3c800822615230c65b4d4cdee3bc7e48cbfa60
  Installing build dependencies ...   Resolved https://github.com/ibm-granite-community/utils to commit da3c800822615230c65b4d4cdee3bc7e48cbfa60
  Installing build dependencies ... -done
  Getting requirements to build wheel ... one
  Getting requirements to build wheel ... -done
  Preparing metadata (pyproject.toml) ... one
  Preparing metadata (pyproject.toml) ... -done
done
Note: 

In [17]:
# Check Python version compatibility
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), \
    f"Python 3.10, 3.11, or 3.12 is required. Current version: {sys.version_info.major}.{sys.version_info.minor}"

print(f"✅ Python version {sys.version_info.major}.{sys.version_info.minor} is compatible.")

✅ Python version 3.12 is compatible.


In [18]:
# Install required packages
%pip install semantic-kernel requests ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 1. Setup Kernel and Granite 3.3 8B Model for Chatbot
import asyncio
import requests
import os
from semantic_kernel import Kernel
from semantic_kernel.prompt_template import PromptTemplateConfig

# Check if we're in a CI environment
is_ci = os.getenv('CI', '').lower() == 'true' or os.getenv('GITHUB_ACTIONS', '').lower() == 'true'

# Initialize kernel
kernel = Kernel()

# Try to import Ollama components - handle gracefully in CI if there are version issues
try:
    from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
    from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
    execution_settings = OllamaChatPromptExecutionSettings()
    ollama_imports_ok = True
except Exception as e:
    if is_ci:
        print(f"⚠️  CI Mode: Ollama connector import issue (expected in some CI environments): {e}")
        print("📝 Creating mock execution settings for CI testing")
        # Create a mock settings object for CI mode
        class MockExecutionSettings:
            pass
        execution_settings = MockExecutionSettings()
        ollama_imports_ok = False
    else:
        print(f"❌ Ollama connector import failed: {e}")
        print("Please ensure you have a compatible version of semantic-kernel installed")
        raise

if is_ci:
    print("🤖 CI Environment detected - Running in mock mode for testing")
    print("✅ Semantic Kernel core imports successful")
    if ollama_imports_ok:
        print("✅ Ollama connector imports successful")
    else:
        print("⚠️  Ollama connector imports skipped (using mock settings)")
    print("✅ Mock setup complete - notebook structure validated")
    
else:
    # Normal operation when running locally
    if not ollama_imports_ok:
        print("❌ Cannot proceed without Ollama connector in local mode")
        raise ImportError("Ollama connector required for local execution")
        
    # Validate Ollama is running and has the Granite model
    ollama_host = "http://localhost:11434"
    granite_model = "granite3.3:8b"

    try:
        # Check if Ollama is running
        response = requests.get(f"{ollama_host}/api/tags", timeout=5)
        response.raise_for_status()
        
        # Check if Granite model is available
        models = response.json()
        available_models = [model['name'] for model in models['models']]
        
        if granite_model not in available_models:
            print(f"❌ Granite model '{granite_model}' not found in Ollama.")
            print(f"Available models: {available_models}")
            print(f"Please run: ollama pull {granite_model}")
            raise ValueError(f"Required model {granite_model} not available")
        
        print(f"✅ Ollama is running and {granite_model} model is available")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Cannot connect to Ollama at {ollama_host}")
        print("Please ensure Ollama is installed and running:")
        print("1. Install Ollama from https://ollama.ai")
        print("2. Run 'ollama serve' in terminal")
        print(f"3. Run 'ollama pull {granite_model}' to download the model")
        raise ConnectionError(f"Ollama connection failed: {e}")

    # Initialize Semantic Kernel with Ollama service
    service_id = "ollama"
    kernel.add_service(
        OllamaChatCompletion(
            service_id=service_id,
            host=ollama_host,
            ai_model_id=granite_model,
        )
    )
    print("✅ Semantic Kernel initialized successfully with Granite 3.3 8B")

In [ ]:
# 2. Define Dynamic Prompt Template for Chat (Following Granite 3.3 guidance)
from datetime import datetime

# Get current date for the system prompt
current_date = datetime.now().strftime("%B %d, %Y")

prompt_template = f"""<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.
Today's Date: {current_date}. You are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>{{{{$input}}}}<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>"""

# Create the function configuration
prompt_template_config = PromptTemplateConfig(
    template=prompt_template, 
    name="chat", 
    template_format="semantic-kernel"
)

# Initialize function variable and mock status tracker
function = None
is_function_mock = False

if is_ci:
    # CI mode: Always create mock function for testing
    print("🤖 CI Mode: Creating mock function for testing")
    class MockFunction:
        def __init__(self):
            pass
    function = MockFunction()
    is_function_mock = True
    print("✅ Mock chat function created for CI testing")
    print(f"✅ System prompt template validated for date: {current_date}")
    print("✅ CI Mode: Template validation complete")
else:
    # Local mode: Create real function
    try:
        function = kernel.add_function(
            function_name="chat_function",
            plugin_name="chat_plugin",
            prompt_template_config=prompt_template_config,
            prompt_execution_settings=execution_settings,
        )
        is_function_mock = False  # Mark as real function using separate variable
        print("✅ Chat function configured with Granite 3.3 prompt template")
        print(f"✅ System prompt includes current date: {current_date}")
        
    except Exception as e:
        print(f"❌ Function creation failed: {e}")
        raise

In [ ]:
# 3. User Input Cell
if is_ci:
    # In CI mode, use a default input for testing
    user_input = "What are the benefits of using Semantic Kernel?"
    print("🤖 CI Mode: Using default test input")
    print(f"Test input: {user_input}")
else:
    # Interactive mode for local usage
    print("💬 Chat with Granite 3.3 8B via Semantic Kernel!")
    print("Example prompts:")
    print("- 'What are the benefits of using Semantic Kernel?'")
    print("- 'Explain quantum computing in simple terms'")
    print("- 'Write a Python function to calculate fibonacci numbers'")
    print()

    user_input = input("Enter your message: ")

In [ ]:
# 4. Define Chat Function with Async Support
import textwrap
import time

async def main():
    try:
        if is_ci:
            # CI mode: Always simulate execution
            print(f"🤖 CI Mode: Simulating chat execution")
            print(f"User input: {user_input}")
            
            # Mock response for testing
            mock_response = """Microsoft Semantic Kernel offers several key benefits:

1. **Unified Framework**: Provides a consistent way to integrate AI capabilities across different models and providers.

2. **Plugin Architecture**: Enables modular development where AI functions can be chained with traditional code.

3. **Multi-Model Support**: Works with various AI providers (OpenAI, Azure OpenAI, Hugging Face, Ollama) without vendor lock-in.

4. **Enterprise-Ready**: Built with security, scalability, and enterprise integration in mind.

5. **Orchestration**: Automatically sequences multiple AI operations for complex workflows.

This demonstrates how Semantic Kernel would orchestrate with Granite 3.3 for enterprise AI applications."""

            print(f"\n🤖 Assistant (simulated response):")
            print("=" * 50)
            print(mock_response)
            print("=" * 50)
            print("✅ CI Mode: Notebook execution flow validated successfully")
            
        else:
            # Local mode: Real execution with Ollama
            if function is None:
                raise RuntimeError("Function not initialized - check previous cells for errors")
            
            if is_function_mock:
                raise RuntimeError("Mock function detected in local mode - check Ollama setup")
            
            print(f"🤔 Thinking... (User: {user_input[:50]}{'...' if len(user_input) > 50 else ''})")
            
            start_time = time.time()
            result = await kernel.invoke(function, input=user_input)
            end_time = time.time()
            response_time = end_time - start_time
            
            # Format the output with line wrapping
            response_text = str(result)
            wrapped_text = textwrap.fill(response_text, width=80, break_long_words=False, break_on_hyphens=False)
            
            print(f"\n🤖 Assistant (responded in {response_time:.2f}s):")
            print("=" * 50)
            print(wrapped_text)
            print("=" * 50)
        
    except Exception as e:
        if is_ci:
            print(f"❌ CI Mode: Unexpected error: {e}")
            print("This would help identify issues in automated testing")
        else:
            print(f"❌ Error occurred: {e}")
            print("\n🔧 Troubleshooting:")
            print("1. Ensure Ollama is running: ollama serve")
            print("2. Ensure Granite model is available: ollama pull granite3.3:8b")
            print("3. Check if Ollama is accessible at http://localhost:11434")
            print("4. Try restarting the notebook kernel")
        raise

print("✅ Chat function defined. Run the next cell to execute the chat.")

In [ ]:
# 5. Execute the Chat
# To chat with the assistant:
# 1. Update the user_input variable below with your message
# 2. Run this cell
# 3. For new messages, change user_input and run this cell again

# Update this line with your message:
user_input = "Hello, what is your name?"

print(f"💬 Sending message: {user_input}")
await main()

In [ ]:
# 6. Interactive Chat (Alternative)
# Run this cell for a more interactive chat experience
# You can keep running this cell for continued conversation

if not is_ci:
    user_input = input("💬 Your message: ")
    print(f"You: {user_input}")
    await main()
else:
    print("🤖 CI Mode: Interactive chat not available in CI environment")

## What's Next?

This example demonstrates the basic chat capabilities of Microsoft Semantic Kernel with IBM Granite 3.3. You can extend this further by leveraging Granite 3.3's advanced features:

### ? **Granite 3.3 Advanced Features**
- **Function Calling**: Enable the model to call external tools and APIs using the `<|tool_call|>` format
- **Fill-in-the-Middle (FIM)**: Code completion using `<fim_prefix>`, `<fim_suffix>`, and `<fim_middle>` tags
- **Reasoning Capabilities**: Use `<think></think>` and `<response></response>` tags for step-by-step reasoning
- **Response Length Control**: Add length annotations (`short` or `long`) to control response verbosity
- **RAG Integration**: Enhanced retrieval-augmented generation with 128K context length

### 🔧 **Semantic Kernel Extensions**
- **Plugin Development**: Create custom plugins that combine AI with traditional code
- **Multi-Model Orchestration**: Use different models for different tasks within the same workflow
- **Memory Integration**: Add persistent memory to maintain context across conversations

### 📚 **Additional Resources**
- [Granite 3.3 Prompt Engineering Guide](https://github.com/ibm-granite/granite-3.3-language-models)
- [Microsoft Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Semantic Kernel GitHub Repository](https://github.com/microsoft/semantic-kernel)
- [IBM Granite Models on Hugging Face](https://huggingface.co/collections/ibm-granite/granite-3-0-language-models-6581b4c0c3e2b6bd7fb74779)
- [Ollama Documentation](https://ollama.ai/docs)

### ? **Key Benefits**
- **Enterprise-Ready**: Both SK and Granite 3.3 are designed for production environments
- **Local Deployment**: Run completely offline with Ollama and Granite 3.3
- **Cost Effective**: No API costs when running locally
- **Apache 2.0 Licensed**: Fully open-source with permissive licensing